<a href="https://colab.research.google.com/github/lobobranco96/construindo_classes/blob/main/web_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
class WebCrawler:

    def __init__(self, url, headers):
        self.url = url
        self.header = headers

    def check_website(self):
      response = requests.get(self.url, headers=headers)
      return response.status_code

    def get_data(self):
      try:
        response = requests.get(self.url, headers=headers)
        if response.status_code == 200:
            items = {}
            soup = BeautifulSoup(response.text, 'html.parser').find("div", class_="sc-tYqdw jXGcA container-purchase")

            # Extrair detalhes do produto
            product_detail = soup.find("h1", class_="sc-58b2114e-6 brTtKt").get_text()
            elements = product_detail.split(',')

            # Exibir cada elemento separado
            for index, element in enumerate(elements):
                items[f"product detail | {index}"] = element.strip()
                #print(f"Elemento {index + 1}: {element.strip()}")

            #items['product_name'] = items.pop('product detail | 0')
            split_product = items.pop('product detail | 0').split(" ", 2)

            # As colunas:
            category = split_product[0]  # "Processador"
            brand = split_product[1]     # "AMD"
            product_name = split_product[2]  # "Ryzen 5 5500"
            items['category'] = category
            items['brand'] = brand
            items['product_name'] = product_name

            # Extrair bloco de preços e descontos
            product_block = soup.find("div", class_="sc-a24aba34-3 hSVqxN").get_text(strip=True)

            # Extrair preços e descontos do bloco
            prices = re.findall(r'R\$\s?[\d.,]+', product_block)
            discount = re.search(r'(\d+)%', product_block)

            if prices and len(prices) >= 2 and discount:
                first_price = prices[0].replace("R$", "").strip().replace(".", "").replace(",", ".")
                second_price = prices[1].replace("R$", "").strip().replace(".", "").replace(",", ".")
                discount_value = discount.group(1)

                items['R$ á vista | antes'] = float(first_price)
                items['R$ á vista | depois'] = float(second_price)
                items['Desconto no Pix'] = int(discount_value)

            credit_card_price = soup.find("div", class_="sc-4f698d6c-0 hkfkrb").get_text()
            cartao_credito = re.search(r"R\$\s?([\d,\.]+)", credit_card_price)
            cartao_credito_parcelado = re.search(r"(\d+)x de R\$\s?([\d,\.]+)", credit_card_price)

            if cartao_credito and cartao_credito_parcelado:
                cartao_credito = cartao_credito.group(1)  # '613,54'
                cartao_credito_parcelado = cartao_credito_parcelado.group(0)  # '10x de R$ 61,35'

            items["Cartão de Credito"] = cartao_credito
            items["Parcelamento"] = cartao_credito_parcelado

            return items
        else:
            print(f"Falha ao acessar o site. Status code: {response.status_code}")
      except requests.exceptions.RequestException as e:
          print(f"Erro ao tentar acessar o site: {e}")

In [3]:
headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
urls = ["https://www.kabum.com.br/produto/356695/processador-amd-ryzen-5-5500-3-6ghz-cache-16mb-hexa-core-12-threads-am4-100-100000457box", "https://www.kabum.com.br/produto/520369/processador-amd-ryzen-7-5700x3d-3-0-ghz-4-1ghz-max-turbo-cache-4mb-8-nucleos-16-threads-am4-100-100001503wof", "https://www.kabum.com.br/produto/320799/processador-amd-ryzen-5-5500-3-6ghz-4-2ghz-max-turbo-cache-19mb-am4-sem-video-100-100000457box", "https://www.kabum.com.br/produto/283718/processador-intel-core-i5-12400f-2-5ghz-4-4ghz-max-turbo-cache-18mb-lga-1700-bx8071512400f", "https://www.kabum.com.br/produto/520369/processador-amd-ryzen-7-5700x3d-3-0-ghz-4-1ghz-max-turbo-cache-4mb-8-nucleos-16-threads-am4-100-100001503wof"]

columns = ["product_name", "category", "brand", "product detail | 1", "product detail | 2", "product detail | 3", "product detail | 4", "product detail | 5", "product detail | 6", "R$ á vista | antes", "R$ á vista | depois", "Desconto no Pix", "Cartão de Credito", "Parcelamento"]
dataframe = pd.DataFrame(columns=columns)
for url in urls:
  try:
    status = WebCrawler(url, headers).check_website()
    if status == 200:
      web = WebCrawler(url, headers)
      item = web.get_data()
      df = pd.DataFrame([item])
      dataframe = pd.concat([dataframe, df],ignore_index=True)
    else:
      print(f"Falha ao acessar o site. Status code: {status}")
  except:
    print("Erro ao tentar acessar o site")

dataframe['Desconto ao preço Antigo'] = dataframe["R$ á vista | antes"] - dataframe['R$ á vista | depois']


<ipython-input-3-9db3787d38f4>:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, df],ignore_index=True)


In [4]:
display(dataframe)

,product_name,category,brand,product detail | 1,product detail | 2,product detail | 3,product detail | 4,product detail | 5,product detail | 6,R$ á vista | antes,R$ á vista | depois,Desconto no Pix,Cartão de Credito,Parcelamento,Desconto ao preço Antigo
0,Ryzen 5 5500,Processador,AMD,3.6GHz,Cache 16MB,Hexa Core,12 Threads,AM4 - 100-100000457BOX,NaN,NaN,NaN,NaN,"689,00","10x de R$ 68,90",NaN
1,Ryzen 7 5700X3D,Processador,AMD,3.0 GHz,(4.1GHz Max Turbo),Cache 4MB,8 Núcleos,16 Threads,AM4 - 100-100001503WOF,2351.13,1659.99,12,"1.886,35","10x de R$ 188,63",691.14
2,Ryzen 5 5500,Processador,AMD,3.6GHz (4.2GHz Max Turbo),Cache 19MB,AM4,Sem Vídeo - 100-100000457BOX,NaN,NaN,NaN,NaN,NaN,"882,34","10x de R$ 88,23",NaN
3,Core i5-12400F,Processador,Intel,2.5GHz (4.4GHz Max Turbo),Cache 18MB,LGA 1700 - BX8071512400F,NaN,NaN,NaN,1058.81,829.99,15,"976,46","10x de R$ 97,64",228.82
4,Ryzen 7 5700X3D,Processador,AMD,3.0 GHz,(4.1GHz Max Turbo),Cache 4MB,8 Núcleos,16 Threads,AM4 - 100-100001503WOF,2351.13,1659.99,12,"1.886,35","10x de R$ 188,63",691.14
